Đây là thuật toán giảm chiều hiện đại , nhanh hơn t-SNE , được giữ cả cấu trúc cục bộ hỗn hợp toàn địa phương , và có thể sử dụng đầu vào cho mô hình ML — đang thay thế t-SNE trong nhiều ứng dụng thực tế!

Mình sẽ trình bày đúng phong cách mà bạn yêu thích:

🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Tương tự t-SNE: giữ mối quan hệ láng khi giảm
Nhưng tốt hơn :

Nhanh hơn
Giữ toàn bộ cấu trúc (khoảng cách giữa các cụm có ý nghĩa)
Có cơ sở toán học vững chắc (dựa trên lý thuyết đa dạng và cấu trúc liên kết )
🧠 Ý tưởng chính — “Bản đồ hóa đa tạp đều say!”
Giả sử dữ liệu nằm trên một bề mặt cong (đa tạp) trong không gian chiều cao (ví dụ: mặt cầu, phạm vi Möbius).
UMAP:
Xây dựng sơ đồ trong không gian gốc: mỗi điểm kết nối với **hàng xóm gầnhàng xóm gần nhất .
Tìm cách nhúng đồ thị vào sao chiều không gian thấp (2D/3D) cho:
Các điểm gần nhau → vẫn gần
Các điểm xa → vẫn xa (ở mức độ hợp lý)
→ Khác với t-SNE chỉ quan tâm "ai gần ai" , UMAP cố gắng giữ hình dạng tổng thể .

✅ Ưu điểm:
Rất nhanh — xử lý hàng trăm điểm dễ dàng.
Giữ toàn bộ cấu trúc → khoảng cách giữa cụm có ý nghĩa .
Có thể dùng để huấn luyện mô hình (vì lập trình ổn định).
Tham số ít, dễ dùng .
❌ Hạn chế:
Thỉnh thoảng ngẫu nhiên (nhưng có ).random_state
Hiệu ứng phụ thuộc vào tham số (nhưng mặc định là tốt).
Ít biến phổ biến trong giấy cũ (t-SNE vẫn được sử dụng nhiều theo thói quen).
🔹 3. Khi nào sử dụng UMAP?
Dữ liệu
lớn
(10k–1M điểm)
Dữ liệu
rất nhỏ
(< 1000 điểm) → PCA/t-SNE đủ
Cần ** giữ cấu t
giữ cấu trúc toàn cục
(khoảng cách giữa cụm từ)
Chỉ cần
trực quan hóa nhanh
và không quan tâm toàn cục
Muốn
sử dụng chiều giảm kết quả để huấn luyện mô hình
Không thể cài đặt thêm thư viện ( )
umap-learn
Làm việc với
nhúng hiện đại
(BERT, CLIP, trình tự protein...)
Có thể
kết quả được xác định hoàn toàn
(UMAP vẫn có ngẫu nhiên)

🎯 So sánh nhanh 3 phép toán giảm chiều:
Tốc độ
⚡⚡⚡ Rất nhanh
🐢 tuột
⚡⚡ Nhanh
Xử lý phi tuyến
❌ Không
✅ Có
✅ Có
Keep cấu trúc cục bộ
Trung bình
✅✅ Rất tốt
✅✅ Rất tốt
Giữ cấu trúc toàn cục
✅ Có
❌ Không
✅ Có
Dữ liệu lớn
✅ Tốt
❌ Kem
✅✅ Rất tốt
Dùng để huấn luyện luyện tập?
✅ Có
❌ Không
✅ Có
Tham số
Nó
Trung bình
Nó

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
import umap

# Tải dữ liệu chữ số (64 chiều)
digits = load_digits()
X = digits.data
y = digits.target

# Chuẩn hóa (nên làm)
X_scaled = StandardScaler().fit_transform(X)

# Áp dụng UMAP (64D → 2D)
umap_model = umap.UMAP(
    n_components=2,
    n_neighbors=15,      # ~ độ cục bộ (giống perplexity của t-SNE)
    min_dist=0.1,        # mức độ tách các điểm trong cụm (0.0 = dồn chặt, 1.0 = tách xa)
    random_state=42
)
X_umap = umap_model.fit_transform(X_scaled)

# Vẽ kết quả
plt.figure(figsize=(12, 5))

# So sánh UMAP vs t-SNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

plt.subplot(1, 2, 1)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', s=8)
plt.title('t-SNE')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='tab10', s=8)
plt.title('UMAP')
plt.axis('off')

plt.suptitle('So sánh t-SNE vs UMAP trên dữ liệu chữ số')
plt.show()

print("✅ UMAP đã chạy thành công!")
print(f"🔍 Dữ liệu: {X.shape[1]}D → UMAP: {X_umap.shape[1]}D")

ModuleNotFoundError: No module named 'umap'